In [1]:
## ctrl + shift + - : 셀 나누기
myRange = spark.range(1000).toDF("number")


# COMMAND ----------

In [2]:
myRange.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



In [3]:
divisBy2 = myRange.where("number % 2 = 0")


# COMMAND ----------

In [4]:
## csv파일 불러오기
flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("../data/flight-data/csv/2015-summary.csv")

# COMMAND ----------

In [5]:
## show를 사용해서 확인하기
flightData2015.describe().show()

+-------+-----------------+-------------------+------------------+
|summary|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|             count|
+-------+-----------------+-------------------+------------------+
|  count|              256|                256|               256|
|   mean|             null|               null|       1770.765625|
| stddev|             null|               null|23126.516918551915|
|    min|          Algeria|             Angola|                 1|
|    max|           Zambia|            Vietnam|            370002|
+-------+-----------------+-------------------+------------------+



In [6]:
flightData2015.createOrReplaceTempView("flight_data_2015")


# COMMAND ----------

In [7]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

sqlWay.explain()
dataFrameWay.explain()


# COMMAND ----------

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#18], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#18, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#18], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#18] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/adminuser/notebooks/My/Spark-book/Spark-The-Definitive-Guide/data/fl..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#18], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#18, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#18], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#18] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/adminuser/notebooks/My/Spark-book/Spark-The-Definitive-Guide/data/fl..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY

In [8]:
from pyspark.sql.functions import max

flightData2015.select(max("count")).take(1)


# COMMAND ----------

[Row(max(count)=370002)]

In [9]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()


# COMMAND ----------

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [10]:
from pyspark.sql.functions import desc

flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()


# COMMAND ----------

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [11]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()


# COMMAND ----------

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#285L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#18,destination_total#285L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#18], functions=[sum(cast(count#20 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#18, 200)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#18], functions=[partial_sum(cast(count#20 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#18,count#20] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/adminuser/notebooks/My/Spark-book/Spark-The-Definitive-Guide/data/fl..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>
